# Sudoku solver
### with genetic algorithm

In [220]:
# imports
import numpy as np

#### Sudoku class

In [221]:
class Sudoku:
    def __init__(self, data: list|np.ndarray, table_size: int, grid_size: int):
        self.size = table_size
        self.nr_values = self.size ** 2
        self.grid_size = grid_size
        self.nr_grid_values = self.grid_size ** 2
        self.nr_grids = self.nr_values // self.nr_grid_values

        if isinstance(data, list):
            data = np.array(data)

        if isinstance(data, np.ndarray):
            self.raw_data = data
            self.rows = np.copy(np.reshape(self.raw_data, (self.size, self.size)))
            self.columns = np.copy(self.rows.T)
            self.grids = np.zeros((self.nr_grids, self.nr_grid_values), np.int32)
            for i in range(self.grid_size):
                for j in range(self.grid_size):
                    xs = (i * self.grid_size) % self.size
                    ys = (j * self.grid_size) % self.size
                    self.grids[i * self.grid_size + j] = np.reshape(self.rows[xs:xs+self.grid_size,ys:ys+self.grid_size], (9,))
            self.fixed_values = None

    def to_rows_idx(self, idx):
        row = idx // self.size
        column = idx % self.size
        return (row, column)

    def to_columns_idx(self, idx):
        row = idx // self.size
        column = idx % self.size
        return (column, row)

    def to_grids_idx(self, idx):
        row = idx // self.size
        column = idx % self.size
        grid = row // self.grid_size * self.grid_size + column // self.grid_size
        nr = row % self.grid_size * self.grid_size + column % self.grid_size
        return (grid, nr)

    def get_rows_data(self, idx):
        return self.rows[self.to_rows_idx(idx)]

    def get_columns_data(self, idx):
        return self.columns[self.to_columns_idx(idx)]

    def get_grids_data(self, idx):
        return self.grids[self.to_grids_idx(idx)]

    def show(self):
        for idx in range(self.nr_values):
            row = idx // self.size + 1
            column = idx % self.size + 1

            if column == 1:
                print(end=" ")

            print(self.raw_data[idx], end=" ")

            if column == self.size:
                print()
                if not row == self.size and row % self.grid_size == 0:
                    print("-" * ((self.size + self.size // self.grid_size - 1) * 2 + 1))
            elif column % self.grid_size == 0:
                print("|", end=" ")

In [222]:
def read_sudoku_from_file(filepath: str) -> Sudoku:
    f = open(filepath, "r")
    size, grid_size = list(map(int, f.readline().split()))
    data = list(map(int, f.read().split()))
    f.close()
    return Sudoku(data, size, grid_size)

In [223]:
sudoku = read_sudoku_from_file("./inputs/input_1.txt")

In [224]:
sudoku.show()

 0 1 0 | 2 0 5 | 0 8 0 
 0 0 3 | 0 0 0 | 4 0 0 
 2 0 0 | 0 7 0 | 0 0 3 
-----------------------
 0 4 0 | 6 1 2 | 0 7 0 
 0 0 0 | 0 0 0 | 0 0 0 
 0 7 0 | 3 9 8 | 0 2 0 
-----------------------
 4 0 0 | 0 6 0 | 0 0 9 
 0 0 2 | 0 0 0 | 8 0 0 
 0 6 0 | 9 0 1 | 0 4 0 
